In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
'''
Function created to take in GenBank File (as a list) and get data frame.
Function iterates over objects (sequences in the genome) in the file 
to read their features. If the feature type is a CDS (if the sequence 
is a coding region) then all of the qualifiers of that CDS are accessed
and put into a dictionary. The contig ID and contig length are assigned 
keys in the dictionary. 

'''

def get_df(recs):
    assert isinstance(recs, list), "Need a list as input"
    assert isinstance(recs[0], SeqRecord), "The input list should contain SeqRecord objects"
    fdicts = []
    for r in recs:
        for feature in r.features:
            if feature.type=="CDS":
                d = dict(feature.qualifiers)
                for k,v in d.items():
                    if len(v)==1:
                        d[k] = v[0]
                d['contig_id'] = r.id
                d['contig_len'] = len(str(r.seq))
                fdicts.append(d)
    dfa = pd.DataFrame(fdicts) 
    return dfa

'''
The GenBank files for A. Nidulans and A. tubingensis are made into lists
and assigned to variable names.
The list of the Genbank File for each organism is passed into the
function above to get the dataframe. The organism key in the dictionary
created is assigned to the organism's name. 
'''

r_nidulans = list(SeqIO.parse("aspergillus_nidulans.gbk", "genbank"))
df_nidulans = get_df(r_nidulans)
df_nidulans['org'] = "A. nidulans"

r_tubingensis = list(SeqIO.parse("aspergillus_tubingensis.gbk", "genbank"))
df_tubingensis = get_df(r_tubingensis)
df_tubingensis['org'] = "A. tubingensis"

In [2]:
"""
Df_all is assigned to both the data frame variables above for each org.
Df_nidulans is assigned to the number of proteins in the A. nidulans
dataframe. Df_tmp fgets the number of unique contigs in the genome 
and then is used to get the genome size.
"""
df_all = pd.concat([df_nidulans, df_tubingensis])
df_nidulans['protein_id'].size 
df_tmp = df_nidulans[['contig_id', 'contig_len']].drop_duplicates() 
df_tmp['contig_len'].sum() 

30200029

In [3]:
#df_all gets the proteins created by each contig.
df_all['protein_count_by_contig'] = df_all.groupby('contig_id')['protein_id'].transform(pd.Series.nunique)

In [4]:
"""
df_subset creates another dataframe for both organisms together, and
contains info about which org it is, what contigs are present, 
the length of each unique contig, and how many proteins are made from 
each contig.
"""
df_subset = df_all[
    ['org', 'contig_id',
    'contig_len',
    'protein_count_by_contig']
    ].drop_duplicates()

In [5]:
"""
Calling upon the variable df_subset gives a chart representing
the information discussed in the cell above.
"""
df_subset

,org,contig_id,contig_len,protein_count_by_contig
0,A. nidulans,NT_107008.1,1482181,486
486,A. nidulans,NT_107009.1,2200800,689
1175,A. nidulans,NW_101435.1,8226,5
1180,A. nidulans,NT_107005.1,1394314,445
1625,A. nidulans,NT_107012.1,2569337,818
...,...,...,...,...
9291,A. tubingensis,NW_023336285.1,2136001,755
10046,A. tubingensis,NW_023336286.1,798263,259
10305,A. tubingensis,NW_023336287.1,1448135,460
10765,A. tubingensis,NW_023336288.1,231917,65


In [6]:
"""
This code gets the number of proteins total, the total length of 
all the contigs, and how many contigs are present for both organisms.
It then puts this information into a table. 
"""
df_subset.groupby('org').agg(
    all_proteins=('protein_count_by_contig', sum),
    genome_size=('contig_len' , sum),
    total_contig=('contig_id' , len)
)

,all_proteins,genome_size,total_contig
org,,,
A. nidulans,9556,30200029,77
A. tubingensis,11477,35047229,15


In [7]:
"""
This function gets the biosyntethic gene cluster information.
A list (GenBank File converted to list) is read and if an object 
of a feature is a protocluster, then the product of the protocluster
is assigned to the variable tmp. The tmp is then added into a list keeping
track of the protocluster product's or an organism.
"""
def get_bgc_df(records, o):
    results = []
    for r in records:
        for f in r.features:
            if f.type == 'protocluster':
                if f.qualifiers:
                    tmp = f.qualifiers.get('product')
                    if tmp:
                        tmp = tmp[0]
                        results.append({'org': o, 'product': tmp})

    return pd.DataFrame(results)

'''
The Genbank files of A. tubingensis are made into a list and assigned
to a variable. This variable is then passed through the function 
above to get the dataframe describing the protoclusters. The same is 
done for the Genbank file on A. nidulans.
'''
tubingensis = list(SeqIO.parse(
    "aspergillus_tubingensis.gbk", "genbank")
    )
df_tubingensis = get_bgc_df(tubingensis, 'A. tubingensis')

nidulans = list(SeqIO.parse(
    "aspergillus_nidulans.gbk", "genbank")
    )
df_nidulans = get_bgc_df(nidulans, 'A. nidulans')

bgc_df = pd.concat([df_tubingensis, df_nidulans])

In [8]:
"""
bgc_overview is a variable that contains information about each organism and the protocluster products. 
It contains the protein type, and how many of each are made. 
"""
bgc_overview = bgc_df.groupby(['org', 'product']).agg(product_count=('product', len)).reset_index()

bgc_overview

,org,product,product_count
0,A. nidulans,NRPS,13
1,A. nidulans,NRPS-like,11
2,A. nidulans,T1PKS,23
3,A. nidulans,betalactone,3
4,A. nidulans,indole,5
5,A. nidulans,terpene,9
6,A. tubingensis,NRPS,20
7,A. tubingensis,NRPS-like,20
8,A. tubingensis,T1PKS,39
9,A. tubingensis,T3PKS,1
